#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2022, 5, 28)

### Yesterday = Last business day

In [2]:
yesterday = date(2022, 5, 27)
yesterday

datetime.date(2022, 5, 27)

### Restart and Run All Cells

In [3]:
cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1133210,WHART,2022-05-25,2022-05-27,2,10.40,10.50,0.10,0.96%,1,0,10.60,10.40,"1,160,956",2022-05-29,2022-05-29,1


In [5]:
names = tmp["name"]
name = names.to_string(index=False)
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.80,I,0.00,11.10,24.74,0.21,11.70,"10,000",-5,4,"30,000",0,0,P50%,SET


### End of Tables in the process

In [6]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '2022-05-27' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [7]:
df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['diff'] = (df['to_price'] - df['fm_price'])
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.sample(5).style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
name,,,,,,,,,,,
BDMS,2022-05-25,2022-05-27,27.00,25.50,"72,005,194",27.00,25.25,B,SET50,-1.50,-5.56%
TOP,2022-05-26,2022-05-27,56.00,57.00,"8,177,983",57.50,56.50,O,SET50,1.00,1.79%
NER,2022-05-26,2022-05-27,6.85,6.80,"11,610,488",6.90,6.75,I,SET,-0.05,-0.73%
SPRC,2022-05-24,2022-05-27,11.60,11.80,"66,444,541",12.20,11.60,O,SET100,0.20,1.72%
ASK,2022-05-25,2022-05-27,41.25,41.50,"1,380,656",42.25,41.00,O,SET,0.25,0.61%


### Price change up or down more than 3%

In [8]:
#df['bdays'] = np.busday_count(df['fm_date'],df['to_date'])
#df['bdays'] = 1
df['bdays'] = (df['to_date'] - df['fm_date']).dt.days
watch_list = df.loc[
    abs(df.percent) >= 3,
    ['percent','status','fm_price','to_price','market','fm_date','bdays','max_price','min_price']
]
watch_list.sort_values(by="percent",ascending=False).style.format(format_dict)

,percent,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
ORI,4.95%,B,10.10,10.60,SET100,2022-05-25,2,10.80,10.20
AP,3.74%,O,10.70,11.10,SET100,2022-05-23,4,11.40,10.70
BCPG,-3.28%,O,12.20,11.80,SET100,2022-05-20,7,12.20,11.80
BDMS,-5.56%,B,27.00,25.50,SET50,2022-05-25,2,27.00,25.25


In [9]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

watch_list.sort_values(['percent'],ascending=[False]).to_csv(output_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(data_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(box_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(one_file)

### Create daily-sales from sales

In [10]:
df[cols].sample(5).style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
name,,,,,,,,,
AP,2022-05-23,2022-05-27,10.70,11.10,"74,884,820",11.40,10.70,3.74%,O
TTB,2022-05-25,2022-05-27,1.23,1.26,"304,100,691",1.27,1.23,2.44%,O
NER,2022-05-26,2022-05-27,6.85,6.80,"11,610,488",6.90,6.75,-0.73%,I
IVL,2022-05-26,2022-05-27,49.00,49.25,"10,054,653",49.75,48.50,0.51%,B
AIT,2022-05-26,2022-05-27,6.75,6.90,"6,597,787",7.00,6.75,2.22%,O


In [11]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(box_file)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file)

### Create monitors from orders

In [12]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.sample(5)

,trade
name,
SPRC,B
GLOBAL,B
BANPU,B
WHART,S
IVL,B


In [13]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,17
1,S,3


### output_file = iCloud Drive on PC

In [14]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [15]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,17
1,S,3


In [16]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
BDMS,B,SET50
IVL,B,SET50
ORI,B,SET100
IP,I,mai
NER,I,SET
WHART,I,SET
AIT,O,SET
AP,O,SET100
ASK,O,SET


In [17]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [18]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
AIT,B
AP,B
ASK,B
BANPU,B
BCPG,B
BDMS,B
BPP,B
GLOBAL,B
GULF,B


In [19]:
buy_candidates.shape[0]

17

In [20]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
IP,S
NER,S
WHART,S


In [21]:
sell_candidates.shape[0]

3